# RAG_Demo
创建简单的RAG模块，对库中的代码分块嵌入并存入向量数据库ChromaDB中，之后拟使用嵌入搜索。
- 创建ChromaDB
- 读取github代码库
- 对代码进行分块和嵌入，存入ChromaDB
- 完成信息检索逻辑

### 1. 创建ChromaDB
- 选择Embedding模型
- 连接chromaDB

In [18]:
import chromadb
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", cache_folder="./models")

client = chromadb.HttpClient(host="localhost", port=8000)

# 测试数据库是否正确连接
try:
    heartbeat = client.heartbeat()
    print(f"Chroma Heartbeat: {heartbeat}") 
except Exception as e:
    print(f"连接失败: {e}")

vector_store = Chroma(
    collection_name="github_codebase",
    embedding_function=embedding_model,
    client=client
)
print("ChromaDB client connected successfully via Docker!")

Chroma Heartbeat: 1765373567185309171
ChromaDB client connected successfully via Docker!


### 2. 读取github代码库

In [19]:
from langchain_community.document_loaders import GithubFileLoader
import os
from dotenv import load_dotenv
load_dotenv()

# 替换为你自己的仓库，或者一个公开的小仓库
# 注意：如果是私有仓库，需要配置 GITHUB_ACCESS_TOKEN
loader = GithubFileLoader(
    repo="msiemens/tinydb",  # 示例仓库
    access_token=os.getenv("GITHUB_ACCESS_TOKEN"),
    github_api_url="https://api.github.com",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".py") # 先只读 Python 文件试试
)

# 读取代码库
documents = loader.load()
print(f"Loaded {len(documents)} documents")

Loaded 20 documents


分块后存入ChromaDB

In [20]:
from langchain_text_splitters import Language

text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=2000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(documents)

# 存入向量数据库
vector_store.add_documents(splits)



['5fbc0dd1-2d40-4f50-942e-88af2e89af20',
 'ac92c9bf-4091-48d8-8265-5dfef702ad1c',
 '9641e6d1-3f0e-4ea6-bc93-7a0c61ac1fcd',
 '944b52eb-6018-4631-8e7d-209292ee636d',
 '0fa1ddb8-6335-44d1-818a-a0f087904ba7',
 '935400e6-aaee-4e48-8a10-40ed4a7e7fb6',
 '6a9dc65b-18cc-4cbd-981d-2baf52f85016',
 'daa67263-8093-4fca-92f4-9d34d3b2c5d9',
 '3417586e-9c80-4f25-ba15-3118d563cab2',
 '9ccb7cae-7ba6-420d-9f69-1470825d08fc',
 '2197969b-05f9-48ee-a63a-a4959b6252b0',
 '24054c59-bf38-47f9-a1f1-2cccc6287202',
 '47738ec3-37d9-4ec6-9ece-93398e5e0212',
 '511486f3-ac3f-4225-9f31-26de4a82ab2f',
 '83e35c71-fb86-4b30-a433-640295b2e6cd',
 '4355b833-bb79-4d57-b2ec-023e061568ef',
 '52185114-38ff-4104-bdca-61c79e88eaa9',
 'ed651112-2676-4980-9f38-29f3950f12ef',
 'f656a69c-0ade-47ca-b6e2-84d00109a670',
 '26c3b1bf-13e5-46b2-bab0-1b9525091106',
 'e38dff77-8f8a-4c64-a546-0fafe3d810af',
 '191c207b-f9ec-46ad-8b68-4b8315facb16',
 '1a0635f2-6567-4dfc-8720-262920e761f6',
 '6e278507-a8bc-4b35-8b90-36a4770bdc81',
 'a106707e-49f4-

In [21]:
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 20, "fetch_k": 40}
    )

In [22]:
results = retriever.invoke("How to create a TinyDB database and insert data?")

for doc in results:
    print(f"Source: {doc.metadata['source']}")
    print(doc.page_content[0:500])
    print("-----")

Source: https://api.github.com/msiemens/tinydb/blob/master/tinydb/__init__.py
"""
TinyDB is a tiny, document oriented database optimized for your happiness :)

TinyDB stores different types of Python data types using a configurable
storage mechanism. It comes with a syntax for querying data and storing
data in multiple tables.

.. codeauthor:: Markus Siemens <markus@m-siemens.de>

Usage example:

>>> from tinydb import TinyDB, where
>>> from tinydb.storages import MemoryStorage
>>> db = TinyDB(storage=MemoryStorage)
>>> db.insert({'data': 5})  # Insert into '_default' tab
-----
Source: https://api.github.com/msiemens/tinydb/blob/master/tinydb/database.py
"""
This module contains the main component of TinyDB: the database.
"""
from typing import Dict, Iterator, Set, Type

from . import JSONStorage
from .storages import Storage
from .table import Table, Document
from .utils import with_typehint

# The table's base class. This is used to add type hinting from the Table
# class to TinyDB. 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 初始化 LLM (DeepSeek)
llm = ChatOpenAI(
    model="deepseek-chat", 
    temperature=0.1 # 代码问题建议低温度，更严谨
)

# 2. 定义 Prompt 模板
# 技巧：在 System Prompt 中明确设定角色和语言要求
template = """你是一个精通 Python 严格的资深代码审查者。请根据以下检索到的代码片段（Context）回答用户的问题。

【要求】
1. 必须使用**中文**回答。
2. 回答时可以使用激进的语言风格，但必须专业且有理有据。
3. 回答应简洁明了，避免冗长。
4. 最后根据评分准则，输出对这个代码库的评分，分数格式"1/100"。
5. 不使用markdown格式。

【评分准则】
- 框架设计（30分）：代码结构是否清晰，模块划分是否合理，是否易于扩展和维护。
- 数据处理（30分）：数据加载、预处理、存储等环节是否高效且符合最佳实践。
- 代码质量（20分）：代码是否遵循PEP8规范，是否有冗余代码，变量命名是否清晰。
- 文档和注释（10分）：是否有足够的文档说明和代码注释，帮助理解代码逻辑。
- 系统安全性（10分）：是否考虑了潜在的安全问题，如输入验证、错误处理等。

【代码片段】
{context}

【用户问题】
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

# 3. 定义格式化文档的辅助函数
def format_docs(docs):
    return "\n\n".join([
        f"--- 文件: {doc.metadata['source']} ---\n{doc.page_content}" 
        for doc in docs
    ])

# 4. 构建 RAG Chain
# 流程：检索 -> 格式化文档 -> 填充 Prompt -> 调用 LLM -> 解析输出
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 5. 运行测试
query = "全局评价这个代码库。"
print(f"正在思考问题: {query} ...\n")

response = rag_chain.invoke(query)
print(response)

正在思考问题: 全局评价这个代码库。 ...

**代码库评价：TinyDB**

这代码库就是个典型的"小而美"玩具数据库，设计思路清晰但架构简陋得令人发指。让我用手术刀给你解剖一下：

### 🔥 核心问题
1. **框架设计（25/30）**  
   - 优点：模块划分清晰（storage/table/query分离），扩展性不错（可自定义storage、document类）
   - 硬伤：全局状态管理薄弱，`_next_id`这种缓存机制在并发下会直接爆炸
   - 表级缓存设计还算聪明，但LRU实现太基础

2. **数据处理（18/30）**  
   - JSON存储直接裸奔，连个事务回滚都没有
   - `freeze()`函数处理嵌套结构算亮点，但深度复制性能堪忧
   - 查询缓存用可变lambda直接破防（见test_mutable_callable），这种设计缺陷能进生产代码？

3. **代码质量（15/20）**  
   - PEP8基本合格，但类型注解半吊子（`Any`满天飞）
   - 测试覆盖率看起来不错，但关键路径缺少压力测试
   - `FrozenDict._immutable`用异常拦截修改，这种hack手法太糙

4. **文档注释（8/10）**  
   - Sphinx文档齐全，连主题配置都给了
   - 但核心算法缺少复杂度标注，`LRUCache.move_to_end`的O(n)操作只字不提

5. **系统安全（3/10）**  
   - 零分！JSONStorage连文件锁都没有，多进程写入直接数据损坏
   - 查询注入风险完全没考虑（虽然场景有限）
   - 输入验证？不存在的，字典随便塞

### 💀 致命伤
- **并发安全：** 整个库假设单线程环境，`_query_cache`和`_next_id`都是定时炸弹
- **存储引擎：** JSON文件当数据库用，没WAL、没校验和、没崩溃恢复
- **性能陷阱：** 每次查询全表扫描，`O(n)`复杂度还敢用LRU缓存当卖点？

### ✅ 可取之处
- 查询API设计优雅：`where('field') == value`链式调用很Pythonic
- 协议设计有前瞻性：`QueryLike`用Protocol支持类型检查
- 测试用例质

正在思考问题: 全局评价这个代码库。 ...

**代码库评价：TinyDB**

这代码库就是个典型的"小而美"玩具数据库，设计思路清晰但架构简陋得令人发指。让我用手术刀给你解剖一下：

### 🔥 核心问题
1. **框架设计（25/30）**  
   - 优点：模块划分清晰（storage/table/query分离），扩展性不错（可自定义storage、document类）
   - 硬伤：全局状态管理薄弱，`_next_id`这种缓存机制在并发下会直接爆炸
   - 表级缓存设计还算聪明，但LRU实现太基础

2. **数据处理（18/30）**  
   - JSON存储直接裸奔，连个事务回滚都没有
   - `freeze()`函数处理嵌套结构算亮点，但深度复制性能堪忧
   - 查询缓存用可变lambda直接破防（见test_mutable_callable），这种设计缺陷能进生产代码？

3. **代码质量（15/20）**  
   - PEP8基本合格，但类型注解半吊子（`Any`满天飞）
   - 测试覆盖率看起来不错，但关键路径缺少压力测试
   - `FrozenDict._immutable`用异常拦截修改，这种hack手法太糙

4. **文档注释（8/10）**  
   - Sphinx文档齐全，连主题配置都给了
   - 但核心算法缺少复杂度标注，`LRUCache.move_to_end`的O(n)操作只字不提

5. **系统安全（3/10）**  
   - 零分！JSONStorage连文件锁都没有，多进程写入直接数据损坏
   - 查询注入风险完全没考虑（虽然场景有限）
   - 输入验证？不存在的，字典随便塞

### 💀 致命伤
- **并发安全：** 整个库假设单线程环境，`_query_cache`和`_next_id`都是定时炸弹
- **存储引擎：** JSON文件当数据库用，没WAL、没校验和、没崩溃恢复
- **性能陷阱：** 每次查询全表扫描，`O(n)`复杂度还敢用LRU缓存当卖点？

### ✅ 可取之处
- 查询API设计优雅：`where('field') == value`链式调用很Pythonic
- 协议设计有前瞻性：`QueryLike`用Protocol支持类型检查
- 测试用例质量高：边界条件和异常场景覆盖充分

---

**最终评分：69/100**  

这分数给的是"教学示范价值"——作为展示如何用Python实现基础数据库概念的教材是优秀的，但任何想在生产环境用它的人，建议先检查一下自己的职业保险够不够赔数据损失。